# Data Cleaning in Python

In [25]:
# Basic libraries for ETLs 
import pandas as pd
import numpy as np
import re
from datetime import datetime

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns


In [26]:
# Read the Excel file
df = pd.read_csv('D:\LẺ\CV\H22023\Trusting Social - Data Analytics Test\data\Q3_data.csv')

df.head()

,ID,Timestamp,UI_version,source,messages,customer_satisfaction
0,50424,11/20/2023 6:04,v1,fb_messenger,"{\n ""chatbot"": ""Welcome! If you're look...",not sastisfy
1,50425,11/20/2023 6:11,v1,website,"{\n ""customer"": ""I thought this razor w...",sastisfy
2,50426,11/20/2023 6:14,v1,website,"{\n ""customer"": ""Hey, I need to report ...",sastisfy
3,50427,11/20/2023 6:26,v1.1.2,website,"{\n ""customer"": ""Can you point me towar...",sastisfy
4,50428,11/20/2023 6:26,v2.sp,tiktok,"{\n ""chatbot"": ""Good day! How can I hel...",sastisfy


## Data Cleaning

In [27]:
# check whether ID column is the primary key (unique customers)
print(len(df))
print(df['ID'].nunique())

2016
2016


ID is the primary key &
No duplicated values

In [28]:
# check for null
df.isnull().sum()

ID                          0
Timestamp                   0
UI_version                  0
source                      0
messages                    0
customer_satisfaction    1619
dtype: int64

### Deal with missing values: customer_satisfaction

customer_satisfaction has 1619 null/2016 value.
To prevent losing valuable information and reducing the sample size of the dataset, which can affect the validity and generalizability of the analysis, I do not drop the Null value. Instead, as the missing balance is not random, I replace null value with "no feedback"

In [29]:
len(df['customer_satisfaction'].unique())

4

In [30]:
df['customer_satisfaction'] = df['customer_satisfaction'].fillna('no feedback')
# check after cleaning
df.isnull().sum()

ID                       0
Timestamp                0
UI_version               0
source                   0
messages                 0
customer_satisfaction    0
dtype: int64

### Deal with data types (Have not done yet)

In [20]:
df.dtypes

ID                        int64
Timestamp                object
UI_version               object
source                   object
messages                 object
customer_satisfaction    object
dtype: object

In [1]:
import pandas as pd
from datetime import datetime
import logging

def clean_and_parse_date(data, correct_date_format):
    # Define the possible date formats
    date_formats = [
        "%A, %B %d, %Y, %I:%M:%S %p",  # Example: "Thursday, November 23, 2023, 3:37:48 PM"
        "%d/%m/%Y %H:%M",              # Example: "23/11/2023 15:37"
        "%d/%m/%Y %I:%M %p",           # Example: "23/11/2023 03:37 PM"
        "%m/%d/%Y %H:%M:%S",           # Example: "11/23/2023 15:37:48"
        "%m/%d/%Y %I:%M %p"            # Example: "11/23/2023 03:37 PM"
    ]
    
    def parse_date(date_str):
        for fmt in date_formats:
            try:
                return datetime.strptime(date_str, fmt)
            except ValueError:
                continue
        return None
    
    # Apply parsing to the 'Timestamp' column
    data['ParsedDate'] = data['Timestamp'].apply(parse_date)
    
    # Separate into good and bad dates
    good_dates = data.dropna(subset=['ParsedDate'])
    bad_dates = data[data['ParsedDate'].isna()]
    
    # Clean bad date entries
    def clean_date(date_str):
        # Replace common incorrect characters
        replacements = ['^', '\\', 's', ':', '-']
        for ch in replacements:
            date_str = date_str.replace(ch, '/')
        return date_str
    
    bad_dates['CleanedTimestamp'] = bad_dates['Timestamp'].apply(clean_date)
    bad_dates['ParsedDate'] = bad_dates['CleanedTimestamp'].apply(parse_date)
    
    # Handle error dates
    error_dates = bad_dates[bad_dates['ParsedDate'].isna()]
    if not error_dates.empty:
        logging.warning("All faulty date formatting was not identified. Check output of error_dates.csv for the faulty date formats.")
        error_dates.to_csv('error_dates.csv', index=False)
    
    # Convert dates to the desired format
    if not good_dates.empty:
        good_dates['FormattedDate'] = good_dates['ParsedDate'].dt.strftime(correct_date_format)
    if not bad_dates.empty:
        bad_dates['FormattedDate'] = bad_dates['ParsedDate'].dt.strftime(correct_date_format)
    
    # Concatenate cleaned data
    output = pd.concat([
        good_dates[['FormattedDate'] + [col for col in data.columns if col != 'Timestamp']],
        bad_dates[['FormattedDate'] + [col for col in data.columns if col != 'Timestamp']]
    ])
    
    return output

cleaned_df = clean_and_parse_date(df['Timestamp'], '%Y-%m-%d')


NameError: name 'df' is not defined

In [23]:
print(df['date_parsed'])
df.isnull().sum()

0      NaT
1      NaT
2      NaT
3      NaT
4      NaT
        ..
2011   NaT
2012   NaT
2013   NaT
2014   NaT
2015   NaT
Name: date_parsed, Length: 2016, dtype: datetime64[ns]


ID                          0
Timestamp                   0
UI_version                  0
source                      0
messages                    0
customer_satisfaction       0
date_parsed              2013
dtype: int64